# NomicEmbeddings

这将帮助您开始使用 LangChain 的 Nomic 嵌入模型。有关 `NomicEmbeddings` 功能和配置选项的详细文档，请参阅 [API 参考](https://python.langchain.com/api_reference/nomic/embeddings/langchain_nomic.embeddings.NomicEmbeddings.html)。

## 概览
### 集成详情

import { ItemTable } from "@theme/FeatureTables";

<ItemTable category="text_embedding" item="Nomic" />

## 设置

要访问 Nomic 嵌入模型，您需要创建一个 Nomic 账户，获取 API 密钥，并安装 `langchain-nomic` 集成包。

### 凭证

前往 [https://atlas.nomic.ai/](https://atlas.nomic.ai/) 注册 Nomic 并生成 API 密钥。完成此操作后，请设置 `NOMIC_API_KEY` 环境变量：

In [2]:
import getpass
import os

if not os.getenv("NOMIC_API_KEY"):
    os.environ["NOMIC_API_KEY"] = getpass.getpass("Enter your Nomic API key: ")

为了实现模型调用的自动化追踪，请设置您的 [LangSmith](https://docs.smith.langchain.com/) API 密钥：

In [3]:
# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")

### 安装

LangChain Nomic 集成位于 `langchain-nomic` 包中：

In [2]:
%pip install -qU langchain-nomic

Note: you may need to restart the kernel to use updated packages.


## 实例化

现在我们可以实例化我们的模型对象并生成聊天补全：

In [10]:
from langchain_nomic import NomicEmbeddings

embeddings = NomicEmbeddings(
    model="nomic-embed-text-v1.5",
    # dimensionality=256,
    # Nomic's `nomic-embed-text-v1.5` model was [trained with Matryoshka learning](https://blog.nomic.ai/posts/nomic-embed-matryoshka)
    # to enable variable-length embeddings with a single model.
    # This means that you can specify the dimensionality of the embeddings at inference time.
    # The model supports dimensionality from 64 to 768.
    # inference_mode="remote",
    # One of `remote`, `local` (Embed4All), or `dynamic` (automatic). Defaults to `remote`.
    # api_key=... , # if using remote inference,
    # device="cpu",
    # The device to use for local embeddings. Choices include
    # `cpu`, `gpu`, `nvidia`, `amd`, or a specific device name. See
    # the docstring for `GPT4All.__init__` for more info. Typically
    # defaults to CPU. Do not use on macOS.
)

## 索引和检索

嵌入模型经常在检索增强生成（RAG）流程中使用，既可以作为索引数据的一部分，也可以在之后检索数据。有关更详细的说明，请参阅我们的[RAG 教程](/docs/tutorials/)。

下面将展示如何使用我们上面初始化的 `embeddings` 对象来索引和检索数据。在此示例中，我们将在 `InMemoryVectorStore` 中索引和检索一个示例文档。

In [5]:
# Create a vector store with a sample text
from langchain_core.vectorstores import InMemoryVectorStore

text = "LangChain is the framework for building context-aware reasoning applications"

vectorstore = InMemoryVectorStore.from_texts(
    [text],
    embedding=embeddings,
)

# Use the vectorstore as a retriever
retriever = vectorstore.as_retriever()

# Retrieve the most similar text
retrieved_documents = retriever.invoke("What is LangChain?")

# show the retrieved document's content
retrieved_documents[0].page_content

'LangChain is the framework for building context-aware reasoning applications'

## 直接使用

底层实现中，vectorstore 和 retriever 会分别调用 `embeddings.embed_documents(...)` 和 `embeddings.embed_query(...)` 来为 `from_texts` 和检索 `invoke` 操作中使用的文本（们）创建 embeddings。

您可以直接调用这些方法来为自己的用例获取 embeddings。

### 嵌入单个文本

您可以使用 `embed_query` 来嵌入单个文本或文档：

In [6]:
single_vector = embeddings.embed_query(text)
print(str(single_vector)[:100])  # Show the first 100 characters of the vector

[0.024642944, 0.029083252, -0.14013672, -0.09082031, 0.058898926, -0.07489014, -0.0138168335, 0.0037


### 嵌入多个文本

您可以使用 `embed_documents` 嵌入多个文本：

In [7]:
text2 = (
    "LangGraph is a library for building stateful, multi-actor applications with LLMs"
)
two_vectors = embeddings.embed_documents([text, text2])
for vector in two_vectors:
    print(str(vector)[:100])  # Show the first 100 characters of the vector

[0.012771606, 0.023727417, -0.12365723, -0.083740234, 0.06530762, -0.07110596, -0.021896362, -0.0068
[-0.019058228, 0.04058838, -0.15222168, -0.06842041, -0.012130737, -0.07128906, -0.04534912, 0.00522


## API 参考

有关 `NomicEmbeddings` 功能和配置选项的详细文档，请参阅[API 参考](https://python.langchain.com/api_reference/nomic/embeddings/langchain_nomic.embeddings.NomicEmbeddings.html)。